In [7]:
!wget https://github.com/thedenaas/hse_seminars/raw/master/2019/seminar_6/data.zip

--2019-12-02 20:20:23--  https://github.com/thedenaas/hse_seminars/raw/master/2019/seminar_6/data.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/thedenaas/hse_seminars/master/2019/seminar_6/data.zip [following]
--2019-12-02 20:20:24--  https://raw.githubusercontent.com/thedenaas/hse_seminars/master/2019/seminar_6/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1064698 (1.0M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   1.01M  1.82MB/s    in 0.6s    

2019-12-02 20:20:26 (1.82 MB/s) - ‘data.zip’ saved [1064698/1064698]



In [8]:
!unzip data.zip
!cp data/ner_short.csv .

Archive:  data.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/ner_short.csv      
  inflating: __MACOSX/data/._ner_short.csv  
  inflating: __MACOSX/._data         


# Assignment 4: Named entity recognition

Create a model for Named Entity Recognition for dataset CoNLL 2002.  
Your quality metric = f1_macro

In your solution you should use: RandomForest, Gradient Boosting (xgboost, lightgbm, catboost)   
Tutorials:  
1. https://github.com/Microsoft/LightGBM/tree/master/examples/python-guide
1. https://github.com/catboost/tutorials 

More baselines you beat - better your score
 
baseline 1 [3 points]: 0.0604      random labels  
baseline 2 [5 points]: 0.3966      PoS features + logistic regression  
baseline 3 [8 points]: 0.8122      word2vec cbow embedding + baseline 2 + svm    

[1 point] using feature engineering (creating features not presented in the baselines)

! Your results must be reproducible. You should explicitly set all seeds random_states in yout model.  
! Remember to use proper training pipeline.  

bonus, think about:  
1. [1 point] Why did we select f1 score with macro averaging as our classification quality measure? What other metrics are suitable?   

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


SEED=1337

In [15]:
df = pd.read_csv('ner_short.csv', index_col=0)
df.head()

,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,sentence_idx,word,tag
0,NNS,demonstrators,IN,of,NNS,__START1__,__START2__,__START2__,__START1__,1.0,Thousands,O
1,VBP,have,NNS,demonstrators,IN,NNS,__START1__,__START1__,Thousands,1.0,of,O
2,VBN,marched,VBP,have,NNS,IN,NNS,Thousands,of,1.0,demonstrators,O
3,IN,through,VBN,marched,VBP,NNS,IN,of,demonstrators,1.0,have,O
4,NNP,London,IN,through,VBN,VBP,NNS,demonstrators,have,1.0,marched,O


In [16]:
null_value_stats = df.isna().sum(axis=0)
null_value_stats[null_value_stats != 0]

Series([], dtype: int64)

In [17]:
# number of sentences
df.sentence_idx.max()

1500.0

In [18]:
# class distribution
df.tag.value_counts(normalize=True)

O        0.852828
B-geo    0.027604
B-gpe    0.020935
B-org    0.020247
I-per    0.017795
B-tim    0.016927
B-per    0.015312
I-org    0.013937
I-geo    0.005383
I-tim    0.004247
B-art    0.001376
I-gpe    0.000837
I-art    0.000748
B-eve    0.000628
I-eve    0.000508
B-nat    0.000449
I-nat    0.000239
Name: tag, dtype: float64

In [0]:
# sentence length
tdf = df.set_index('sentence_idx')
tdf['length'] = df.groupby('sentence_idx').tag.count()
df = tdf.reset_index(drop=False)

Firstly, let's take a glance on our targets, so we can hopefully notice some commmon features between them:

In [20]:
df.loc[df.tag != "O"].sample(20)

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length
62087,1288.0,WDT,which,",",",",NNP,NNP,CD,1998,Noumea,Accord,I-tim,78
9265,422.0,VBD,said,RB,also,NNP,IN,NN,spokeswoman,in,Islamabad,B-geo,66
31531,1414.0,",",",",NNP,Rhine-Westphalia,NNP,IN,NN,state,of,North,B-geo,78
59082,1161.0,VBZ,has,NNP,II,NNP,NNP,NNP,Pope,John,Paul,I-per,66
36555,140.0,VBN,asked,VBZ,has,NNP,NNP,VBD,said,General,Adam,I-per,76
1937,84.0,JJ,Sudanese,VBD,accused,NNP,IN,NNS,forces,in,Darfur,B-geo,50
45175,536.0,NNP,Saadoun,",",",",NN,NNP,DT,A,Sunni,Arab,I-gpe,58
46673,598.0,DT,another,VBD,added,CD,IN,NNP,Trinidad,in,1910,B-tim,26
5687,259.0,NNP,Council,NNP,Security,NNP,NNP,DT,the,United,Nations,I-org,58
65989,1460.0,VBN,met,VBZ,has,NNP,NNP,NNP,Prosecutor,Patrick,Fitzgerald,I-org,46


Alright, so the target almost unilaterally start either with a capital letter or a numeric. This of course can be true also for sentence starts, but we'll leave to the model to distinguish that. Let's add these two binary features -- data engineering, for hell sake!

In [0]:
def starts_with_capital(word):
  if not word:
    return 0.0
  return int(word[0].upper() == word[0] and word[0].lower() != word[0])

def starts_with_number(word):
  if not word:
    return 0.0
  return int(ord(word[0]) in range(ord("0"), ord("9")+1))

df['capital_first'] = df.apply(lambda row: starts_with_capital(row.word), axis=1)
df['number_first'] = df.apply(lambda row: starts_with_number(row.word), axis=1)

Let's check that:

In [22]:
df.loc[df.capital_first == 1].sample(5)

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length,capital_first,number_first
7025,320.0,JJ,next,IN,in,NNP,NNP,NNP,President,Yoweri,Museveni,I-per,58,1,0
6557,300.0,NNS,officials,NNP,House,NNP,__START1__,__START2__,__START2__,__START1__,White,B-org,44,1,0
51511,811.0,VBP,have,NNS,migrants,JJ,CD,IN,than,60,African,B-gpe,38,1,0
5753,262.0,NNPS,Nations,NNP,United,DT,__START1__,__START2__,__START2__,__START1__,The,O,60,1,0
8086,370.0,",",",",NNP,Hagino,NNP,__START1__,__START2__,__START2__,__START1__,Yu,B-per,42,1,0


In [23]:
df.loc[df.number_first == 1].sample(5)

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length,capital_first,number_first
45375,544.0,.,.,NN,percent,CD,VB,TO,to,hit,7.5,O,42,0,1
17902,804.0,",",",",NNS,tsunamis,CD,NNP,DT,the,December,26,I-tim,64,0,1
57144,1072.0,NNP,Mexico,IN,in,CD,NNP,VBZ,performs,March,17,I-tim,20,0,1
1638,70.0,WDT,that,NNS,economies,CD,DT,IN,of,the,21,O,42,0,1
16247,732.0,IN,from,CD,billion,CD,$,VB,raise,$,1.5,O,60,0,1


This should give us **[1 point] for using feature engineering**

Now, let's look how the data is currently presented in the DataFrame:

In [24]:
df.loc[0]

sentence_idx                  1
next-next-pos               NNS
next-next-word    demonstrators
next-pos                     IN
next-word                    of
pos                         NNS
prev-pos             __START1__
prev-prev-pos        __START2__
prev-prev-word       __START2__
prev-word            __START1__
word                  Thousands
tag                           O
length                       48
capital_first                 1
number_first                  0
Name: 0, dtype: object

In [25]:
df.head()

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length,capital_first,number_first
0,1.0,NNS,demonstrators,IN,of,NNS,__START1__,__START2__,__START2__,__START1__,Thousands,O,48,1,0
1,1.0,VBP,have,NNS,demonstrators,IN,NNS,__START1__,__START1__,Thousands,of,O,48,0,0
2,1.0,VBN,marched,VBP,have,NNS,IN,NNS,Thousands,of,demonstrators,O,48,0,0
3,1.0,IN,through,VBN,marched,VBP,NNS,IN,of,demonstrators,have,O,48,0,0
4,1.0,NNP,London,IN,through,VBN,VBP,NNS,demonstrators,have,marched,O,48,0,0


In order to use LightGBM, let's encode categorial variables (so it will encode them afterwards using its gimmicks):

In [0]:
le = LabelEncoder()
df['pos'] = le.fit_transform(df['pos'])
df['next-pos'] = le.fit_transform(df['next-pos'])
df['next-next-pos'] = le.fit_transform(df['next-next-pos'])
df['prev-pos'] = le.fit_transform(df['prev-pos'])
df['prev-prev-pos'] = le.fit_transform(df['prev-prev-pos'])
df['word'] = le.fit_transform(df['word'])
df['next-word'] = le.fit_transform(df['next-word'])
df['next-next-word'] = le.fit_transform(df['next-next-word'])
df['prev-word'] = le.fit_transform(df['prev-word'])
df['prev-prev-word'] = le.fit_transform(df['prev-prev-word'])
df['tag'] = le.fit_transform(df['tag'])

Train-test splitting:

In [48]:
y = df.tag
Ndf = df.drop(columns=['tag'])

df_train, df_test, y_train, y_test = model_selection.train_test_split(Ndf, y, stratify=y, 
                                                                      test_size=0.25, random_state=SEED, shuffle=True)
print('train', df_train.shape[0])
print('test', df_test.shape[0])

train 50155
test 16719


Finally, let's define and train the model:

In [0]:
X_train = df_train
X_test = df_test

In [59]:
num_train, num_feature = X_train.shape

# create dataset for lightgbm
# if you want to re-use data, remember to set free_raw_data=False
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train, free_raw_data=False)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 17,
    'metric': 'multi_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

feature_name = ["sentence_idx", "next-next-pos", "next-next-word", "next-pos",
                "next-word", "pos", "prev-pos", "prev-prev-pos",
                "prev-prev-word", "prev-word", "word", "length",
                "capital_first", "number_first"]

categorical_feature = ["next-next-pos", "next-next-word", "next-pos",
                "next-word", "pos", "prev-pos", "prev-prev-pos",
                "prev-prev-word", "prev-word", "word",
                "capital_first", "number_first"]

print('Starting training...')
# feature_name and categorical_feature
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=5000,
                early_stopping_rounds=10,
                valid_sets=lgb_train,  # eval training data
                feature_name=feature_name,
                categorical_feature=categorical_feature)

Starting training...
[1]	training's multi_logloss: 0.623104
Training until validation scores don't improve for 10 rounds.
[2]	training's multi_logloss: 0.561878
[3]	training's multi_logloss: 0.51764
[4]	training's multi_logloss: 0.481741
[5]	training's multi_logloss: 0.452418
[6]	training's multi_logloss: 0.426806
[7]	training's multi_logloss: 0.403427
[8]	training's multi_logloss: 0.383223
[9]	training's multi_logloss: 0.365496
[10]	training's multi_logloss: 0.348794
[11]	training's multi_logloss: 0.333315
[12]	training's multi_logloss: 0.318778
[13]	training's multi_logloss: 0.305576
[14]	training's multi_logloss: 0.293313
[15]	training's multi_logloss: 0.281972
[16]	training's multi_logloss: 0.271188
[17]	training's multi_logloss: 0.261288
[18]	training's multi_logloss: 0.25185
[19]	training's multi_logloss: 0.243074
[20]	training's multi_logloss: 0.23489
[21]	training's multi_logloss: 0.226983
[22]	training's multi_logloss: 0.219659
[23]	training's multi_logloss: 0.212581
[24]	trai

And now let's 

In [60]:
print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

Saving model...
Starting predicting...


Convert predictions to our required format:

In [0]:
y_pred = [list(o).index(max(o)) for o in y_pred]

And let's run the test:

In [72]:
print('test', metrics.f1_score(list(y_test), y_pred, average='macro'))

test 0.8787403280193746


This is better than `baseline 3: 0.8122 word2vec cbow embedding + baseline 2 + svm`, so we should get **8 points** for the model and **9 points** in total.